In [1]:
import json

In [105]:
def import_file(path):
    with open(path, "r") as file:
        c = json.load(file)
        file.close()
        return c
    
def create_relations(annotations):
    """
    For every image, creates a list of triples in the form of (s, o, p)
    where s and o are the indexes of the annotations within the image and p is the index of the relation_category
    """

def get_bbox_relation(a, b):
    """
    Simple heuristic to return a rel_cat id to two bounding boxes
    """
    a_xyxy = xywh_to_xyxy(a)
    b_xyxy = xywh_to_xyxy(b)
    if box_similarity(a_xyxy, b_xyxy) > 0.6:
        return 0
    elif box_similarity(a_xyxy, b_xyxy) > 0.3:
        if a[1] > b[1]: # When a is below b
            return 1
        else:
            return 2
    else:
        
    return None
    
def xywh_to_xyxy(a):
    return (a[0] + a[2], a[0], a[1] + a[3], a[1])

def box_similarity(a, b):
    """
    Returns fraction of overlap with b to own box. 1 if perfect overlap, 0 if no overlap
    a == b == [xmax, xmin, ymax, ymin]
    """
    a_a = (a[0] - a[1]) * (a[2] - a[3])
    a_b = (b[0] - b[1]) * (b[2] - b[3])
    overlap = max(min(a[0], b[0]) - max(a[1], b[1]), 0) * \
              max(min(a[2], b[2]) - max(a[3], b[3]), 0)

    if a_a > a_b:
        return overlap / a_a
    else:
        return overlap / a_b

rel_cat = ["on", "below", "above"]

In [98]:
val = import_file("val.json")
rel = import_file("rel.json")
train = import_file("train.json")

In [22]:
image2ann = dict()
for a in train["annotations"]:
    image_id = a["image_id"]
    if image_id in image2ann.keys():
        image2ann[image_id].append(a)
    else:
        image2ann[image_id] = [a]

In [95]:
# For every image, examine their annotiosw
image_rel = dict()
for img, ann_list in image2ann.items():
    triples = []
    for i, a in enumerate(ann_list):
        for j, b in enumerate(ann_list):
            if i >= j:
                continue
            rel = get_bbox_relation(a["bbox"], b["bbox"])
            if rel:
                triples.append([i, j, rel])
    image_rel[img] = triples

In [73]:
xywh_to_xyxy(train["annotations"][0]["bbox"])

(404, 363, 628, 602)

In [102]:
image_rel

{'cl987t0fj1nt7074z8uli8uis': [[1, 5, 1]],
 'cl987t0fa1nk7074zhwnv2j60': [],
 'cl987t0fi1ns3074z9smhhmll': [[8, 9, 1],
  [8, 10, 1],
  [8, 11, 1],
  [9, 10, 1],
  [9, 11, 1],
  [10, 11, 1]],
 'cl987t0fb1nlz074z6v529i8b': [],
 'cl74qgww3096g0878akhf4z3l': [],
 'cl74qgwvo08oo08781tld9y6e': [[4, 5, 2]],
 'cl74qgww809cw08789shn1vs4': [],
 'cl5l1gjbb130r075jg4ucf16o': [[3, 10, 3], [6, 7, 3], [7, 8, 3], [8, 9, 1]],
 'cl74qgwvj08jg0878gicoh01a': [],
 'cl74qgwwd09ks08788lpb5s5c': [[0, 16, 3]],
 'cl5l1gjbd136j075j7eai6k8x': [],
 'cl74qgwvr08rc08787m6327u8': [],
 'cl5l1gjbf13cv075jeg659iqc': [],
 'cl5l1gjbg13fz075jgq7jcjcx': [[3, 4, 2], [4, 5, 2], [5, 6, 2]],
 'cl74qgww909fg08786j2aedmz': [[4, 9, 2]],
 'cl987t0fa1nk3074ze7h3che8': [],
 'cl74qgwvj08jc08789rtoglys': [],
 'cl987t0fi1nsr074zbycv32tb': [[5, 12, 1]],
 'cl74qgwvo08os08780kde5isb': [[1, 2, 2]],
 'cl987t0fi1nrz074z5z1bhkvs': [[0, 1, 1],
  [0, 2, 1],
  [0, 3, 1],
  [1, 2, 1],
  [1, 3, 1],
  [2, 3, 1]],
 'cl5l1gjbf13cr075j2xv7ayq3': [],
 '

In [106]:
out_rel = {"train": image_rel, "rel_categories": rel_cat}

In [109]:
with open("oic_rel.json", "w") as file:
    json.dump(out_rel, file)
    file.close()